# Get Top Performing Stations

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
import warnings
from glob import glob
from itertools import product
from typing import List

import altair as alt
import duckdb
import pandas as pd
from tqdm.contrib import concurrent as concurrent_tq
from watermark import watermark

/opt/conda/envs/eda/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
_ = alt.data_transformers.disable_max_rows()
_ = alt.renderers.set_embed_options(actions=False)

In [4]:
PROJ_ROOT = os.pardir
src_dir = os.path.join(PROJ_ROOT, "src")
sys.path.append(src_dir)

In [5]:
%aimport file_utils
import file_utils as flut

%aimport pandas_utils
import pandas_utils as pu

%aimport visualization_helpers
import visualization_helpers as vzu

## About

### Objective

The processed bike share ridership data is explored in order to identify the top-performing bike share stations using historical bike share ridership data.

### Data

The following previously-created datasets are used in this exploratory data analysis

1. list of downtown neighbourhoods
2. station info (includes geodata)
3. processed bike share ridership

### Assumptions

1. Same as in data retrieval and processing steps.

### Outputs

1. (1 file) Performance ranks for all currently active stations and as indication of whether station is a top-performer or not, with a filename of the format `stations_performance__YYYYmmdd_HHMMSS.parquet.gzip`.

## User Inputs

In [6]:
# ridership
years_proc_trips = {
    2018: [f'Q{k}' for k in range(1, 4+1)],
    2019: [f'Q{k}' for k in range(1, 4+1)],
    2020: [f'{str(k).zfill(2)}' for k in range(1, 12+1)],
    2021: [f'{str(k).zfill(2)}' for k in range(1, 12+1)],
    2022: [f'{str(k).zfill(2)}' for k in range(1, 12+1)],
    2023: [f'{str(k).zfill(2)}' for k in range(1, 3+1)],
}

# top-performing stations
last_full_year = 2022
# # selection threshold to get top-performers
top_perform_frac = 0.16

# exporting to disk
my_timezone = 'America/Toronto'

In [7]:
data_dir = os.path.join(PROJ_ROOT, 'data')
raw_data_dir = os.path.join(data_dir, 'raw', 'systems', 'toronto')
processed_data_dir = os.path.join(data_dir, 'processed')
reports_dir = os.path.join(PROJ_ROOT, 'reports')
figures_dir = os.path.join(reports_dir, 'figures')

# processed trips
fpaths_proc = {
    y: [
        f
        for p in periods
        for f in sorted(
            glob(
                os.path.join(
                    processed_data_dir,
                    f'processed__trips_{y}_{p}*.parquet.gzip',
                )
            )
        )
    ]
    for y, periods in years_proc_trips.items()
}
fpaths_proc_all = [f for _, v in fpaths_proc.items() for f in v]
fpaths_proc_2018_2022 = [f for y in range(2018, 2022+1) for f in fpaths_proc[y]]

# downtown neighbourhoods
fpath_downtown_neighs = glob(
    os.path.join(raw_data_dir, 'downtown_neighbourhoods__*.parquet.gzip')
)[0]

# station info for currently active stations
fpath_stations_info = glob(
    os.path.join(raw_data_dir, 'stations_info__*.parquet.gzip')
)[0]

In [8]:
def run_sql_query(query: str, verbose: bool=False) -> pd.DataFrame:
    """Run SQL query using DuckDB."""
    with warnings.catch_warnings():
        warnings.simplefilter("ignore", FutureWarning)
        df_query = duckdb.sql(query).df()
    if verbose:
        print(f"Query returned {len(df_query):,} rows")
    return df_query


def run_parallel(
    inputs_product: product,
    fn,
    chunk_size: int=100,
) -> pd.DataFrame:
    """Run function against multiple inputs in parallel."""
    iterables = list(inputs_product)
    outputs = list(
        concurrent_tq.process_map(
            fn,
            *zip(*iterables),
            max_workers=12,
            chunksize=chunk_size,
        )
    )
    return outputs

## Get Data

### Downtown Neighbourhoods

Show previously retrieved neighbourhoods within downtown Toronto

In [9]:
df_downtown_neighs = pd.read_parquet(fpath_downtown_neighs)
df_downtown_neighs

,Neighbourhood,Location,is_downtown
0,University,Downtown,True
1,Kensington-Chinatown,Downtown,True
2,Wellington Place,Downtown,True
3,Harbourfront-CityPlace,Downtown,True
4,Bay-Cloverhill,Downtown,True
5,Yonge-Bay Corridor,Downtown,True
6,St Lawrence-East Bayfront-The Islands,Downtown,True
7,Church-Wellesley,Downtown,True
8,Downtown Yonge East,Downtown,True
9,North St.James Town,Downtown,True


### Bike Share Station Info (MetaData)

Show the stations info data that was retrieved previously, containing station name and its associated neighbourhood name

In [11]:
%%time
query = f"""
        SELECT station_id,
               name,
               physical_configuration,
               capacity,
               is_charging_station,
               rental_methods LIKE '%CREDITCARD%' AS credit,
               Neighbourhood,
               COALESCE(Location, NULL, 'Others') AS Location,
               COALESCE(is_downtown, NULL, False) AS is_downtown,
               census_tract_id
        FROM read_parquet({[fpath_stations_info]})
        LEFT JOIN df_downtown_neighs USING (Neighbourhood)
        -- WHERE physical_configuration <> 'VAULT'
        ORDER BY station_id, name
        """
df_info = run_sql_query(query).convert_dtypes()
with pd.option_context('display.max_columns', None):
    pu.show_df(df_info)

column,station_id,name,physical_configuration,capacity,is_charging_station,credit,Neighbourhood,Location,is_downtown,census_tract_id
dtype,string[python],string[python],string[python],Int64,boolean,boolean,string[python],string[python],boolean,string[python]
nunique,790,790,6,40,2,2,107,4,2,272
missing,0,0,0,0,0,0,0,0,0,0
0,7000,Fort York Blvd / Capreol Ct,REGULAR,35,False,True,Harbourfront-CityPlace,Downtown,True,5350012.01
1,7001,Wellesley Station Green P,ELECTRICBIKESTATION,23,True,True,Church-Wellesley,Downtown,True,5350063.06
2,7002,St. George St / Bloor St W,REGULAR,19,False,True,University,Downtown,True,5350061.00
3,7003,Madison Ave / Bloor St W,REGULAR,15,False,True,Annex,Others,False,5350091.01
4,7005,King St W / York St,REGULAR,23,False,True,Yonge-Bay Corridor,Downtown,True,5350014.00
...,...,...,...,...,...,...,...,...,...,...
785,7926,McRae Dr / Laird Dr - SMART,SMARTMAPFRAME,24,False,False,Leaside-Bennington,Others,False,5350195.02


CPU times: user 25.3 ms, sys: 125 µs, total: 25.4 ms
Wall time: 24.1 ms


### Processed Bike Share Ridership Data

Show the file of processed bike share ridership for August of 2022

In [12]:
%%time
query = f"""
        SELECT *
        FROM read_parquet({[fpaths_proc[2022][7]]})
        WHERE started_at_year = 2022
        AND started_at_month = 8
        LIMIT 3
        """
df_proc_trips_preview = run_sql_query(query).convert_dtypes()
pu.show_df(df_proc_trips_preview)

column,trip_id,start_station_id,started_at,start_station_name,end_station_id,ended_at,end_station_name,bike_id,user_type,started_at_year,started_at_month,started_at_day,started_at_hour,started_at_minute,ended_at_year,ended_at_month,ended_at_day,ended_at_hour,ended_at_minute
dtype,Int64,Int32,datetime64[us],string[python],Int64,datetime64[us],string[python],Int64,string[python],Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32,Int32
nunique,3,2,1,2,2,2,1,3,1,1,1,1,1,1,1,1,1,1,2
missing,0,0,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,0
0,17515458,7259,2022-08-01,Lower Spadina Ave / Lake Shore Blvd,7712,2022-08-01 00:20:00,<NA>,3328,Casual Member,2022,8,1,0,0,2022,8,1,0,20
1,17515440,7208,2022-08-01,80 Clinton St North of College,7534,2022-08-01 00:07:00,Walnut Ave / Queen St W,4662,Casual Member,2022,8,1,0,0,2022,8,1,0,7
2,17515442,7259,2022-08-01,Lower Spadina Ave / Lake Shore Blvd,7712,2022-08-01 00:20:00,<NA>,4510,Casual Member,2022,8,1,0,0,2022,8,1,0,20


CPU times: user 51.1 ms, sys: 3.87 ms, total: 54.9 ms
Wall time: 76 ms


**Notes**

1. The above contents come from the file containing processed bike share ridership for August of 2022. The file with processed data for all other months (2020, 2021, 2022 and 2023) and quarters (2018, 2019) contain the same column names.

## Identify Top-Performing Stations

### Get Top `N` Stations Based on Specified Threshold

Get the number of wanted top stations to be separated from all other stations based on their performance in each of the following metrics

1. total departures during the last **full** year (2022)
2. total arrivals during the last **full** year (2022)
3. total departures over the last `X` **full** years (2018 to 2022)
4. total arrivals over the last `X` **full** years (2018 to 2022)

In [13]:
num_top_stations = int(top_perform_frac*len(df_info))
print(
    f"Selected stations that rank in the top {num_top_stations:,} "
    f"in four metrics, out of {len(df_info):,}"
)

Selected stations that rank in the top 126 in four metrics, out of 790


**Notes**

1. Here
   - `X` will be chosen as 5, which corresponds to all historical **full** years since 2018 inclusive
   - based on the assumptions earlier, the last **full** year is 2022

### Get Top-Performers Overall (Including Metadata)

Get the top-performing stations using the following approach with all the processed bike share ridership data

1. get departures and number of stations used in departures during the most recent full year (2022)
2. get arrivals and number of stations used in arrivals during the most recent full year (2022)
3. get departures and number of stations used in departures during the last 5 full years (2018 to 2022, inclusive)
4. get arrivals and number of stations used in arrivals during the last 5 full years (2018 to 2022, inclusive)
5. get station attributes
6. Combine (`UNION`) the departures and arrivals totals (1,2,3,4)
7. reshape (6) into untidy data so that departures and arrivals are now columns
8. combine (`INNER JOIN`) with the station attributes for currently active stations (5) and assign ranks based on the four metrics defined above, namely
   - departures in last full year (2018)
   - arrivals in last full year (2018)
   - departures overall (2018 to 2022, inclusive)
   - arrivals overall (2018 to 2022, inclusive)
9. Use the following rule-based logic and ranks assigned above to indicate if a station is a top-performing station
   - **top-performing stations are in the top 100 in each of the following criteria**
     - **departures in last full year**
     - **departures overall**
     - **arrivals in last full year**
     - **arrivals overall**

In [13]:
%%time
query = f"""
        -- 1. get total departures and number of stations from most recent full year (2022)
        WITH t1 AS (
            SELECT start_station_id AS station_id,
                   COUNT(DISTINCT(trip_id)) AS trips_last_year,
                   'departures' As type
            FROM read_parquet({fpaths_proc[2022]})
            GROUP BY all
        ),
        -- 2. get total arrivals and number of stations from most recent full year (2022)
        t2 AS (
            SELECT end_station_id AS station_id,
                   COUNT(DISTINCT(trip_id)) AS trips_last_year,
                   'arrivals' As type
            FROM read_parquet({fpaths_proc[2022]})
            GROUP BY all
        ),
        -- 3. get total departures and number of stations from N most-recent full years (2018 to 2022)
        t3 AS (
            SELECT start_station_id AS station_id,
                   COUNT(DISTINCT(trip_id)) AS trips_last_n_years
            FROM read_parquet({fpaths_proc_2018_2022})
            GROUP BY all
        ),
        -- 4. get total arrivals and number of stations from N most-recent full years (2018 to 2022)
        t4 AS (
            SELECT end_station_id AS station_id,
                   COUNT(DISTINCT(trip_id)) AS trips_last_n_years
            FROM read_parquet({fpaths_proc_2018_2022})
            GROUP BY all
        ),
        -- 5. get useful station attributes (excludes any single-value attributes since these
        -- are not useful for extracting insights)
        t5 AS (
            SELECT station_id,
                   name,
                   physical_configuration,
                   capacity,
                   is_charging_station,
                   rental_methods LIKE '%CREDITCARD%' AS credit,
                   Neighbourhood,
                   COALESCE(Location, NULL, 'Others') AS Location,
                   COALESCE(is_downtown, NULL, False) AS is_downtown,
                   census_tract_id,
                   True as is_active
            FROM read_parquet({[fpath_stations_info]})
            LEFT JOIN df_downtown_neighs USING (Neighbourhood)
        ),
        -- 6. combine all
        t6 AS (
            SELECT *
            FROM t1 LEFT JOIN t3 USING (station_id)
            UNION ALL
            SELECT *
            FROM t2 LEFT JOIN t4 USING (station_id)
        ),
        -- 7. reshape into untidy data
        t7 AS (
            PIVOT t6
            ON type USING MAX(trips_last_year) AS last_year, MAX(trips_last_n_years) AS last_n_years
        ),
        -- 8. filter to get active stations, fill NULLs and assign ranks based on departures & arrivals
        t8 AS (
            SELECT * EXCLUDE (
                       is_downtown,
                       departures_last_year,
                       arrivals_last_year,
                       departures_last_n_years,
                       arrivals_last_n_years,
                   ),
                   COALESCE(departures_last_year, NULL, 0) AS departures_last_year,
                   COALESCE(arrivals_last_year, NULL, 0) AS arrivals_last_year,
                   COALESCE(departures_last_n_years, NULL, 0) AS departures_last_n_years,
                   COALESCE(arrivals_last_n_years, NULL, 0) AS arrivals_last_n_years,
                   COALESCE(is_downtown, NULL, False) AS is_downtown,
                   RANK() OVER(ORDER BY departures_last_year DESC) AS rank_deps_last_year,
                   RANK() OVER(ORDER BY departures_last_n_years DESC) AS rank_deps_last_n_years,
                   RANK() OVER(ORDER BY arrivals_last_year DESC) AS rank_arrs_last_year,
                   RANK() OVER(ORDER BY arrivals_last_n_years DESC) AS rank_arrs_last_n_years,
            FROM t7
            INNER JOIN t5 USING (station_id)
        ),
        -- 9. append conditional to indicate if station is top-performing station
        t9 AS (
            SELECT *,
                   (
                       CASE WHEN (
                           rank_deps_last_year <= {num_top_stations}
                           AND rank_arrs_last_year <= {num_top_stations}
                           AND rank_deps_last_n_years <= {num_top_stations}
                           AND rank_arrs_last_n_years <= {num_top_stations}
                       ) THEN True ELSE False END
                   ) AS is_top_perform_station
            FROM t8
        )
        SELECT *
        FROM t9
        """
df_stations = run_sql_query(query).convert_dtypes()
with pd.option_context('display.max_columns', None):
    pu.show_df(df_stations)

column,station_id,name,physical_configuration,capacity,is_charging_station,credit,Neighbourhood,Location,census_tract_id,is_active,departures_last_year,arrivals_last_year,departures_last_n_years,arrivals_last_n_years,is_downtown,rank_deps_last_year,rank_deps_last_n_years,rank_arrs_last_year,rank_arrs_last_n_years,is_top_perform_station
dtype,Int64,string[python],string[python],Int64,boolean,boolean,string[python],string[python],string[python],boolean,Int64,Int64,Int64,Int64,boolean,Int64,Int64,Int64,Int64,boolean
nunique,627,627,6,39,2,2,83,4,214,1,611,603,616,618,2,611,616,603,618,2
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
0,7076,York St / Queens Quay W,REGULAR,57,False,True,Harbourfront-CityPlace,Downtown,5350012.04,True,38593,43978,142416,158202,True,1,1,1,1,True
1,7016,Bay St / Queens Quay W (Ferry Terminal),REGULAR,35,False,True,St Lawrence-East Bayfront-The Islands,Downtown,5350013.02,True,27451,28291,109448,117385,True,3,2,5,2,True
2,7033,Union Station,REGULAR,43,False,True,St Lawrence-East Bayfront-The Islands,Downtown,5350013.01,True,23752,32254,85757,110893,True,14,13,2,3,True
3,7175,HTO Park (Queens Quay W),REGULAR,27,False,True,Harbourfront-CityPlace,Downtown,5350012.04,True,25078,29431,98277,110699,True,9,6,4,4,True
4,7203,Bathurst St/Queens Quay(Billy Bishop Airport),REGULAR,35,False,True,Fort York-Liberty Village,West of Downtown,5350008.02,True,22130,25285,100580,109788,False,23,5,15,5,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,7736,Birchmount Rd / Kingston Rd,VAULT,15,False,False,Birchcliffe-Cliffside,Others,5350338.00,True,7,8,7,8,False,623,623,622,622,False


CPU times: user 20.2 s, sys: 1.23 s, total: 21.4 s
Wall time: 2.08 s


**Notes**

1. In step 5. the `LEFT JOIN` introduces missing values. In step 8., these are filled with zeroes.
2. In step 8., the `INNER JOIN` is required in order to select stations that were used in ridreship between 2018 and 2022 (inclusive) that are also still active. Inactive stations are not relevant for the marketing campaign.
3. The most recent full year was chosen as one criteria to select stations that captured recent patterns in ridership.
4. The overall ridership was chosen as the second criteria in order to select stations that have a history of high bike share demand. Stations that have only come online recently might have garnered interest due to their novelty but are not consistently capable of generating high bike share demand. The client's campaign should pick reliable stations in order to maximize awareness of the MCU continuing education program. For this reason, a presence of high overall demand was the second criteria chosen to identify top-performing stations.
5. The top 100 currently active stations corresponds to the top 16% of all currently active stations. For this reason `top_perform_frac` was set to 16%.

**Summary of Metrics Used**

1. total departures during the last **full** year (2022)
2. total arrivals during the last **full** year (2022)
3. total departures over the last `X` **full** years (2018 to 2022)
4. total arrivals over the last `X` **full** years (2018 to 2022)

### Get Top-Performers on Weekdays and Weekends Separately (Excluding Metadata)

Get the top-performing stations on weekdays and weekens using the same approach as above, but using only weekday or weekend trips from the processed bike share ridership data

In [14]:
%%time
df_stations_intra_week = {}
for k, dow in zip(['weekday', 'weekend'], [[0,1,2,3,4], [5,6]]):
    dow_str = ','.join([str(d) for d in dow])
    query = f"""
            -- 1. get total departures and number of stations from most recent full year (2022)
            WITH t1 AS (
                SELECT start_station_id AS station_id,
                       COUNT(DISTINCT(trip_id)) AS trips_last_year,
                       'departures' As type
                FROM read_parquet({fpaths_proc[2022]})
                WHERE ISODOW(started_at)-1 IN ({dow_str})
                GROUP BY all
            ),
            -- 2. get total arrivals and number of stations from most recent full year (2022)
            t2 AS (
                SELECT end_station_id AS station_id,
                       COUNT(DISTINCT(trip_id)) AS trips_last_year,
                       'arrivals' As type
                FROM read_parquet({fpaths_proc[2022]})
                WHERE ISODOW(started_at)-1 IN ({dow_str})
                GROUP BY all
            ),
            -- 3. get total departures and number of stations from N most-recent full years (2018 to 2022)
            t3 AS (
                SELECT start_station_id AS station_id,
                       COUNT(DISTINCT(trip_id)) AS trips_last_n_years
                FROM read_parquet({fpaths_proc_2018_2022})
                WHERE ISODOW(started_at)-1 IN ({dow_str})
                GROUP BY all
            ),
            -- 4. get total arrivals and number of stations from N most-recent full years (2018 to 2022)
            t4 AS (
                SELECT end_station_id AS station_id,
                       COUNT(DISTINCT(trip_id)) AS trips_last_n_years
                FROM read_parquet({fpaths_proc_2018_2022})
                WHERE ISODOW(started_at)-1 IN ({dow_str})
                GROUP BY all
            ),
            -- 5. get useful station attributes (excludes any single-value attributes since these
            -- are not useful for extracting insights)
            t5 AS (
                SELECT station_id,
                       name
                FROM read_parquet({[fpath_stations_info]})
                LEFT JOIN df_downtown_neighs USING (Neighbourhood)
            ),
            -- 6. combine all
            t6 AS (
                SELECT *
                FROM t1 LEFT JOIN t3 USING (station_id)
                UNION ALL
                SELECT *
                FROM t2 LEFT JOIN t4 USING (station_id)
            ),
            -- 7. reshape into untidy data
            t7 AS (
                PIVOT t6
                ON type
                USING MAX(trips_last_year) AS {k}_last_year,
                      MAX(trips_last_n_years) AS {k}_last_n_years
            ),
            -- 8. fill NULLs and assign ranks based on departures and arrivals
            t8 AS (
                SELECT * EXCLUDE (
                           departures_{k}_last_year,
                           arrivals_{k}_last_year,
                           departures_{k}_last_n_years,
                           arrivals_{k}_last_n_years,
                       ),
                       COALESCE(departures_{k}_last_year, NULL, 0) AS departures_{k}_last_year,
                       COALESCE(arrivals_{k}_last_year, NULL, 0) AS arrivals_{k}_last_year,
                       COALESCE(departures_{k}_last_n_years, NULL, 0) AS departures_{k}_last_n_years,
                       COALESCE(arrivals_{k}_last_n_years, NULL, 0) AS arrivals_{k}_last_n_years,
                       RANK() OVER(ORDER BY departures_{k}_last_year DESC) AS rank_{k}_deps_last_year,
                       RANK() OVER(ORDER BY departures_{k}_last_n_years DESC) AS rank_{k}_deps_last_n_years,
                       RANK() OVER(ORDER BY arrivals_{k}_last_year DESC) AS rank_{k}_arrs_last_year,
                       RANK() OVER(ORDER BY arrivals_{k}_last_n_years DESC) AS rank_{k}_arrs_last_n_years,
                FROM t7
                INNER JOIN t5 USING (station_id)
            ),
            -- 9. append conditional to indicate if station is top-performing station
            t9 AS (
                SELECT *,
                       (
                           CASE WHEN (
                               rank_{k}_deps_last_year <= {num_top_stations}
                               AND rank_{k}_arrs_last_year <= {num_top_stations}
                               AND rank_{k}_deps_last_n_years <= {num_top_stations}
                               AND rank_{k}_arrs_last_n_years <= {num_top_stations}
                           ) THEN True ELSE False END
                       ) AS is_top_perform_station_{k}
                FROM t8
            )
            SELECT *
            FROM t9
            """
    df_stations_intra_week[k] = run_sql_query(query).convert_dtypes()
df_stations_weekdays, df_stations_weekends = list(df_stations_intra_week.values())

CPU times: user 36.7 s, sys: 1.04 s, total: 37.7 s
Wall time: 3.61 s


### Combine Overall, Weekday & Weekend Top-Performers (Including Metadata)

Use a `LEFT JOIN` on the `station_id` and `name` columns to combine all the above station performances

In [15]:
query = f"""
        SELECT *
        FROM df_stations
        LEFT JOIN df_stations_weekdays USING (station_id, name)
        LEFT JOIN df_stations_weekends USING (station_id, name)
        """
df_stations_combo = run_sql_query(query).convert_dtypes()
with pd.option_context('display.max_columns', None):
    pu.show_df(df_stations_combo)

column,station_id,name,physical_configuration,capacity,is_charging_station,credit,Neighbourhood,Location,census_tract_id,is_active,departures_last_year,arrivals_last_year,departures_last_n_years,arrivals_last_n_years,is_downtown,rank_deps_last_year,rank_deps_last_n_years,rank_arrs_last_year,rank_arrs_last_n_years,is_top_perform_station,departures_weekday_last_year,arrivals_weekday_last_year,departures_weekday_last_n_years,arrivals_weekday_last_n_years,rank_weekday_deps_last_year,rank_weekday_deps_last_n_years,rank_weekday_arrs_last_year,rank_weekday_arrs_last_n_years,is_top_perform_station_weekday,departures_weekend_last_year,arrivals_weekend_last_year,departures_weekend_last_n_years,arrivals_weekend_last_n_years,rank_weekend_deps_last_year,rank_weekend_deps_last_n_years,rank_weekend_arrs_last_year,rank_weekend_arrs_last_n_years,is_top_perform_station_weekend
dtype,Int64,string[python],string[python],Int64,boolean,boolean,string[python],string[python],string[python],boolean,Int64,Int64,Int64,Int64,boolean,Int64,Int64,Int64,Int64,boolean,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,boolean,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64,boolean
nunique,627,627,6,39,2,2,83,4,214,1,611,603,616,618,2,611,616,603,618,2,606,594,614,609,606,614,594,609,2,570,576,601,598,570,601,576,598,2
missing,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,1,1,1,1,1,1,1,0,0,0,0,0,0,0,0,0
0,7076,York St / Queens Quay W,REGULAR,57,False,True,Harbourfront-CityPlace,Downtown,5350012.04,True,38593,43978,142416,158202,True,1,1,1,1,True,24425,27958,92962,103596,2,1,1,1,True,14168,16020,49454,54606,1,2,1,1,True
1,7016,Bay St / Queens Quay W (Ferry Terminal),REGULAR,35,False,True,St Lawrence-East Bayfront-The Islands,Downtown,5350013.02,True,27451,28291,109448,117385,True,3,2,5,2,True,16903,17456,69306,74888,13,4,13,4,True,10548,10835,40142,42497,5,5,5,5,True
2,7033,Union Station,REGULAR,43,False,True,St Lawrence-East Bayfront-The Islands,Downtown,5350013.01,True,23752,32254,85757,110893,True,14,13,2,3,True,17726,24900,68404,90677,6,5,2,2,True,6026,7354,17353,20216,35,46,20,33,True
3,7175,HTO Park (Queens Quay W),REGULAR,27,False,True,Harbourfront-CityPlace,Downtown,5350012.04,True,25078,29431,98277,110699,True,9,6,4,4,True,13872,15490,56437,63065,31,14,20,9,True,11206,13941,41840,47634,3,4,2,3,True
4,7203,Bathurst St/Queens Quay(Billy Bishop Airport),REGULAR,35,False,True,Fort York-Liberty Village,West of Downtown,5350008.02,True,22130,25285,100580,109788,False,23,5,15,5,True,13160,15270,61724,68428,37,9,21,6,True,8970,10015,38856,41360,8,6,8,6,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
622,7156,Salem Ave / Bloor St W,REGULAR,15,False,True,Dovercourt Village,Others,5350096.02,True,5364,5230,20884,20181,False,276,241,282,244,False,3709,3656,14688,14325,279,237,282,242,False,1655,1574,6196,5856,276,232,282,246,False


**Observations**

1. All the performance stats on weekdays have missing values. Overall and weekend performance stats do not have missing values. This indicates that some stations were only used on weekends (not on weekdays).

### Show Fraction of Market Penetration As the Number of Top-Performers Increases

Iterate over the desired number of top-performers and perform the following

1. Append the following columns are appended to the output of the above query (per station)
   - cumulative number of trips from top-performing stations
   - total number of trips
   - for each station get the fraction of trips (market penetration) from top-performers relative to total number of trips
2. Select the `N` top-performing stations and then get the maximum market penetration for this selection

In [19]:
def get_frac_ridership_by_top_n_stations(
    num_stations: int, num_stations_selected: int
) -> pd.DataFrame:
    """."""
    query = f"""
            -- 1. append columns and calculate market penetration per station
            WITH t1 AS (
                SELECT *,
                       -- append running total of departures (trips) during last year
                       SUM(departures_last_year) OVER(ORDER BY departures_last_year DESC) AS trips_last_year_cumsum,
                       -- append running fraction of departures (trips) during last year
                       SUM(departures_last_year) OVER() AS total_trips_last_year,
                       -- calculate market penetration metric
                       100* trips_last_year_cumsum/total_trips_last_year AS frac_trips_last_year
                FROM df_stations_combo
            ),
            -- 2. selectop top-N stations and calculate maximum market penetration
            t2 AS (
                SELECT 2022 AS year,
                       MAX(frac_trips_last_year) AS frac_trips_last_year,
                       {num_stations} AS num_stations,
                       {num_stations_selected} AS num_stations_selected
                FROM t1
                WHERE rank_deps_last_year <= {num_stations}
            )
            SELECT *
            FROM t2
            """
    df = run_sql_query(query).convert_dtypes()
    return df

In [20]:
%%time
outputs = run_parallel(
    product(range(10, len(df_stations), 2), [df_stations['is_top_perform_station'].sum()]),
    get_frac_ridership_by_top_n_stations,
    chunk_size=24,
)
df_stations_frac_market_penetration = (
    pd.concat(outputs, ignore_index=True).convert_dtypes()
)
with pd.option_context('display.max_columns', None):
    pu.show_df(df_stations_frac_market_penetration)

100%|██████████| 309/309 [00:00<00:00, 454.07it/s]


column,year,frac_trips_last_year,num_stations,num_stations_selected
dtype,Int32,Float64,Int32,Int32
nunique,1,308,309,1
missing,0,0,0,0
0,2022,6.529967,10,98
1,2022,7.675537,12,98
2,2022,8.808799,14,98
3,2022,9.925813,16,98
4,2022,11.020201,18,98
...,...,...,...,...
304,2022,99.99879,618,98


CPU times: user 87.9 ms, sys: 148 ms, total: 235 ms
Wall time: 904 ms


Show the above output as a plot

In [21]:
%%time
chart = vzu.plot_line_chart(
    df_stations_frac_market_penetration,
    xvar = 'num_stations:Q',
    yvar = "frac_trips_last_year:Q",
    xvar_line = 'num_stations_selected:Q',
    xtitle = 'Number of Top-Performing Stations',
    ytitle = 'Fraction of 2022 Trips (%)',
    line_color = 'darkgreen',
    vertical_line_color = 'red',
    annotation_text = 'Chosen Number of Top-Performers',
    annotation_text_color = 'red',
    annotation_text_opacity = 0.5,
    annotation_text_loc = dict(dx=250, dy=25),
    axis_label_fontsize = 14,
    title_loc = 45,
    fig_size = dict(width=450, height=270),
)
chart.save(
    os.path.join(figures_dir, '00_market_penetration_by_num_top_stations.png')
)
chart

CPU times: user 538 ms, sys: 59.8 ms, total: 598 ms
Wall time: 823 ms


alt.LayerChart(...)

**Observations**

1. The selected number of top-performers (98) had a ~42% market penetration during 2022. The remaining ~520 stations (520/98 = 5.3X) were required to capture the remaining ~58% (58/42 = 1.4X).
2. Here, the number of top-performers was chosen manually to be 98. An inflection point ([knee](https://en.wikipedia.org/wiki/Knee_of_a_curve)) is not visible in this chart. So, this approach is not capable of fine-tuning the number of top-performers.

## Export to Disk

In [15]:
%%time
fname_prefix = "stations_performance"
_ = df_stations_combo.pipe(
    flut.load,
    processed_data_dir,
    fname_prefix,
    my_timezone,
    True,
)

Exported 627 rows of stations_performance data to /home/jovyan/data/processed/stations_performance__20240312_103503.parquet.gzip
CPU times: user 29.4 ms, sys: 0 ns, total: 29.4 ms
Wall time: 29.1 ms


## Discussion

### Conclusions

1. This step has identified the top performing stations across the Bike Share Toronto network using historical ridership data using the following rank-based metrics
   - departures during most recent full year (2022)
   - arrivals during most recent full year (2022)
   - departures during all full years of data that are available (2018 to 2022)
   - arrivals during all full years of data that are available (2018 to 2022)

## Summary of Assumptions

1. Same as in data retrieval and processing step.

## Version Information

In [22]:
packages = [
    'numpy',
    'pandas',
    'pyarrow',
    'duckdb',
    'altair',
    'vl-convert',
]
print(
    watermark(
        updated=True,
        current_date=True,
        current_time=True,
        timezone=True,
        custom_time="%Y-%m-%d %H:%M:%S %Z",
        python=True,
        machine=True,
        packages=','.join(packages),
    )
)

Last updated: 2024-03-18 04:58:00 UTC

Python implementation: CPython
Python version       : 3.11.8
IPython version      : 8.22.2

numpy     : 1.26.4
pandas    : 2.2.1
pyarrow   : 15.0.1
duckdb    : 0.10.0
altair    : 5.2.0
vl-convert: not installed

Compiler    : GCC 12.3.0
OS          : Linux
Release     : 6.6.10-76060610-generic
Machine     : x86_64
Processor   : x86_64
CPU cores   : 12
Architecture: 64bit

